# Towards a full version of french wino (final steps)

## Why

To make data really comprehensible.

In [95]:
import os,sys,inspect
import pandas as pd
import numpy as np
import importlib
import sys
from utils_xo import *
importlib.reload(sys.modules["utils_xo"])
from utils_xo import *
pd.set_option('display.max_colwidth', None)
# set data directory
data_dir = r"../../../data/french/"
import xml.etree.ElementTree as ET
import re

In [96]:
tree = ET.parse(data_dir + 'winograd_fr_v2.xml')
root = tree.getroot()

#* get id and schema for no 9999 items
ids = []
french_schema_ids = []
for schema in root.findall("./"):
    if schema.attrib['engn'] != "9999":
        ids.append(int(schema.attrib['engn']))
        french_schema_ids.append(int(schema.attrib['id']))

In [97]:
#* make ids and french schema ids twice longer
double_ids = []
double_french_schema_ids = []
for id in ids:
    double_ids.append(id)
    double_ids.append(id+1)
for fr in french_schema_ids:
    double_french_schema_ids.append(fr)
    double_french_schema_ids.append(fr)

In [98]:
print(f'{len(double_ids)} items in french')

202 items in french


In [103]:
#* add text and hypothesis: [texte] il mange trop -> [hypothese] il mange pas peu

#[ gather texts
items_a = []
items_b = []

for schema in root.findall("schema"):
    if schema.attrib['engn'] != "9999":
        text_debut = schema.find(".//txt1").text.lstrip()
        text_middle = schema.find(".//wordA").text
        text_end = schema.find(".//txt2").text
        item_a = text_debut + text_middle + text_end
        text_debut = schema.find(".//txt1").text.lstrip()
        text_middle = schema.find(".//wordB").text
        text_end = schema.find(".//txt2").text
        item_b = text_debut + text_middle +  text_end
        items_a.append(item_a)
        items_b.append(item_b)

In [104]:
items = []
for x,y in zip(items_a,items_b):
    items.append(x)
    items.append(y)

In [105]:
#* add questions and answers, questiona should be followed by question b in the final list, the same for answers, the incorrect answer is the i in answerb or a
questions_a = []
questions_b = []
answers_a = []
answers_b = []
for schema in root.findall("schema"):
    if schema.attrib['engn'] != "9999":
        text_debut = schema.find(".//qn1").text
        text_middle = schema.find(".//qwordA").text
        text_end = schema.find(".//qn2").text
        # print(text_debut,text_middle,text_end)
        if not text_middle:
            text_middle = ""
        if not text_debut:
            text_debut = ""
        question_a = text_debut + text_middle + text_end
        text_debut = schema.find(".//qn1").text
        text_middle = schema.find(".//qwordB").text
        text_end = schema.find(".//qn2").text
        if not text_middle:
            text_middle = ""
        if not text_debut:
            text_debut = ""
        question_b = text_debut + text_middle +  text_end
        # print(text_debut,text_middle,text_end)
        answer_a = schema.find(".//answer1").text
        # print(answer_a)
        answer_b = schema.find(".//answer2").text
        # print(answer_b)
        questions_a.append(question_a)
        questions_b.append(question_b)
        answers_a.append(answer_a)
        answers_b.append(answer_b)
print(len(questions_a))
print(len(questions_b))
print(len(answers_a))
print(len(answers_b))
# 214
# clean the text
for ls in [answers_a,answers_b,questions_a,questions_b]:
    for i,text in enumerate(ls):
        ls[i] = text.strip().replace("  "," ").replace("   ","")

101
101
101
101


In [106]:
#[ make question list and answer list
questions = []
answers = []
for a,b in zip(questions_a,questions_b):
    questions.append(a)
    questions.append(b)
for c,d in zip(answers_a,answers_b):
    answers.append(c)
    answers.append(d)

In [107]:
#! add the wrong answer
#[ gather wrong answers
wrong_answers = []
for x,y in zip(answers_b,answers_a):
    wrong_answers.append(x)
    wrong_answers.append(y)

In [108]:
#! make hypothese and masked hypothese
#[ hypothese
hypotheses = []
masked_hypotheses = []

for schema in root.findall("schema"):
    if schema.attrib['engn'] != "9999":
        text_debut = schema.find(".//answer1").text.lstrip()
        text_middle = schema.find(".//qwordA").text
        if not text_middle:
            text_middle = schema.find(".//qn2").text
        temp = text_middle
        # print(text_debut,text_middle)
        item_a = text_debut + " " + text_middle
        item_a_masked = "<mask>" + " " + text_middle
        masked_hypotheses.append(item_a_masked)
        hypotheses.append(item_a)
        text_debut = schema.find(".//answer2").text.lstrip()
        text_middle = schema.find(".//qwordB").text
        if not text_middle:
            text_middle = temp
        # print(text_debut,text_middle)
        item_a = text_debut + " " + text_middle
        item_a_masked = "<mask>" + " " + text_middle
        masked_hypotheses.append(item_a_masked)
        hypotheses.append(item_a)
print(len(hypotheses))
print(len(masked_hypotheses))
for x,y in zip(hypotheses,masked_hypotheses):
    print(x,y,sep="\n")

202
202
la coupe grand
<mask> grand
la valise petit
<mask> petit
Paul n'a pas réussi
<mask> n'a pas réussi
Georges n'a pas répondu
<mask> n'a pas répondu
l'avocat répéter
<mask> répéter
le témoin répondre à
<mask> répondre à
Nicolas faible
<mask> faible
son fils lourd
<mask> lourd
les lycéens puni
<mask> puni
les collégiens défendu
<mask> défendu
la carafe vide
<mask> vide
la tasse plein
<mask> plein
Bernard généreux
<mask> généreux
Charles reconnaissant
<mask> reconnaissant
Bernard blessé
<mask> blessé
Charles ingrat
<mask> ingrat
l'escroc gagner
<mask> gagner
Samuel perdre
<mask> perdre
le poisson affamé
<mask> affamé
le ver délicieux
<mask> délicieux
l'arbre déplacer
<mask> déplacer
le toit réparer
<mask> réparer
son ancienne maison déménager de
<mask> déménager de
sa nouvelle maison emménager dans
<mask> emménager dans
les gens enthousiaste
<mask> enthousiaste
les livres de Paul populaire
<mask> populaire
Claire déçue
<mask> déçue
Sylvie sortie
<mask> sortie
Claire dehors
<mask> de

In [122]:
#[ add masked text
# read use_reunion10
masked_texts = pd.read_csv(data_dir+"use_reunion10.csv",sep=";")
masked_texts = masked_texts.masked.tolist()
type(masked_texts)#%%

#[ add correct answer head
# read use_reunion10
df_bert = pd.read_csv(data_dir+"use_reunion10.csv",sep=";")
correct_answer_heads = df_bert.answer.tolist()
options = df_bert.options.tolist()
schemas =  df_bert.id_schema.tolist()
corre_answer_heads = []
wrong_answer_heads = []
maske_texts = []
for i,ans in enumerate(correct_answer_heads):
    if schemas[i] not in [29,39,40,56,82,96]:
        option = options[i]
        wrong_answer = option.replace(ans,"").strip()
        wrong_answer_heads.append(wrong_answer)
        corre_answer_heads.append(option)
        maske_texts.append(masked_texts[i])
masked_texts = maske_texts
correct_answer_heads = corre_answer_heads
print(len(correct_answer_heads))
print(len(wrong_answer_heads))
print(len(double_ids))
print(len(double_french_schema_ids))
print(len(questions))
print(len(answers))
print(len(items))
print(len(hypotheses))
print(len(masked_hypotheses))
print(len(masked_texts))

202
202
202
202
202
202
202
202
202
202


In [123]:
#! probleme a regler sur les conseillers et pas que
# build the first final version
dt_df = {"id":double_ids,"french_schema_no": double_french_schema_ids,"question":questions,"correct_answer_head":correct_answer_heads,
         "wrong_answer_head":wrong_answer_heads,
         "correct_answer":answers,"text":items,"wrong_answer":wrong_answers,"hypothesis":hypotheses,"masked_hypothesis":masked_hypotheses,"masked_text":masked_texts}
df = pd.DataFrame(dt_df)
df.shape
df = df[["id","text","masked_text","question","hypothesis","masked_hypothesis","correct_answer","wrong_answer","correct_answer_head","wrong_answer_head","french_schema_no"]]
df.head()
xo_write_json(data_dir+"winofr_final_v2.json",df)
final1 = xo_read_json(data_dir+"winofr_final_v2.json")
#[ reorder the ids
final2 = sorted(final1, key= lambda x: x["id"])
final2
#[ keep only the ones in wsc273
final3 = [x for x in final2 if x["correct_answer"] in ["les conseillers municipaux","les manifestants"] or x["id"] != 0]
# final3
# final3 = [x for x in final3 if x["text"] == "Les conseillers municipaux ont refusé une autorisation aux manifestants, car ils craignent la violence. " or x["id"] != 1]
# final3
# #[ some cleanning
for it in final3:
    it["text"] = it["text"].strip()
    it["question"] = it["question"].strip()
    it["correct_answer"] = it["correct_answer"].strip()
final3
xo_write_json(data_dir+"winofr_final_v3.json",df)
len(final3) #202

202